In [80]:
import keras
from keras.layers import Input, Activation, Dense, Conv2D, MaxPooling2D,Flatten,Lambda
from keras.models import Model
import keras.backend as K
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
import glob
import cv2
import numpy as np
import math

In [24]:
dataset_path="./python/images_background/"
languages = [path.split("/")[-1] for path in glob.glob(dataset_path+"*")]
all_alphabets = {language:[cv2.cvtColor(cv2.imread(image),cv2.COLOR_RGB2GRAY) for image in glob.glob(dataset_path+language+"/*/*")]\
                                                                       for language in languages }
all_images =  np.vstack([np.array(all_alphabets[key]) for key in all_alphabets])

In [ ]:
digit_input = Input(shape=(105,105,1))

x = Conv2D(64, (10, 10),activation='relu',input_shape=(105,105))(digit_input)
x = MaxPooling2D()(x)
x = Conv2D(128, (8, 8),activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(128, (6, 6),activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(256, (4, 4),activation='relu')(x)
out = Flatten()(x)

#out = Dense(4096,activation='sigmoid')(x)
vision_model = Model(digit_input,out)

letter1 = Input(shape=(105,105,1))
letter2 = Input(shape=(105,105,1))

out_1 = vision_model(letter1)
out_2 = vision_model(letter2)

def manhattan_distance(A,B):
    return K.sum(K.abs(A-B),axis=1,keepdims=True)

merged_vector = Lambda(lambda x:manhattan_distance(x[0],x[1]), output_shape=lambda inp_shp:(inp_shp[0][0],1))([out_1,out_2])

out_fin = Dense(4096,activation="sigmoid")(merged_vector)
final_output = Dense(1,activation="sigmoid")(out_fin)

final_model = keras.models.Model(inputs=[letter1, letter2], outputs=final_output)

In [32]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))
        print('lr:', step_decay(len(self.losses)))
def step_decay(epoch):
    initial_lrate = 1e-4
    lrate = initial_lrate * math.pow(0.99,epoch)
    return lrate
loss_history = LossHistory()
lrate = LearningRateScheduler(step_decay)
callbacks_list = [loss_history, lrate]

optimizer = SGD(lr=1e-4,momentum=0.5,decay=0,nesterov=False)

In [83]:
final_model.compile(optimizer="adam",
                   loss='binary_crossentropy',
                   metrics=['binary_accuracy'],)

In [34]:
final_model.layers[2].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 105, 105, 1)       0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 96, 96, 64)        6464      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 41, 41, 128)       524416    
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 15, 15, 128)       589952    
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 7, 7, 128)         0         
__________

In [84]:
def generator(features,batch_size):
    # Create empty arrays to contain batch of features and labels#
    lengthes = np.cumsum([len(all_alphabets[key]) for key in all_alphabets])  
    batch_features = np.zeros((batch_size, 2, 105, 105,1))
    batch_labels = np.zeros((batch_size,1))
    while True:
        for i in range(batch_size):
            indecies= np.random.choice(len(all_images),2)
            #print(indecies)
            image1,image2 = all_images[indecies[0]].reshape(105,105,1),all_images[indecies[1]].reshape(105,105,1)
            batch_features[i] = (image1,image2)
            distance1,distance2 = lengthes-indecies[0],lengthes-indecies[1]
            not_insame_class = np.argmax(distance1>0) != np.argmax(distance2>0)
            batch_labels[i] = not_insame_class
        yield [batch_features[:,0],batch_features[:,1]],batch_labels              

In [72]:
batch = next(generator(all_images,1))

In [87]:
final_model.fit_generator(generator(all_images,128),
                          steps_per_epoch=5,
                          verbose=2,
                          epochs=100,
                          callbacks=callbacks_list)

Epoch 1/100
 - 63s - loss: 0.1748 - binary_accuracy: 0.9578
lr: 9.900000000000001e-05
Epoch 2/100
 - 64s - loss: 0.1751 - binary_accuracy: 0.9578
lr: 9.801e-05
Epoch 3/100
 - 65s - loss: 0.1802 - binary_accuracy: 0.9562
lr: 9.70299e-05
Epoch 4/100


KeyboardInterrupt: 